<a href="https://colab.research.google.com/github/akashmathur-2212/LLMs-playground/blob/main/LlamaIndex-applications/Advanced-RAG/reranker_models_evaluation/LLM_rerankers_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reranker models

This notebook showcases how to do a two-stage pass for retrieval. Use `embedding-based` retrieval with a high `top-k` value
in order to maximize recall and get a large set of candidate items. Then, use `LLM-based` retrieval
to dynamically select the nodes that are actually relevant to the query.

In [1]:
!pip install -qqq llama-index llama-hub cohere langchain openai accelerate==0.21.0 bitsandbytes==0.40.2 transformers sentence_transformers InstructorEmbedding

In [12]:
pip install llama-index-postprocessor-cohere-rerank

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install llama-index-embeddings-huggingface

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install llama-index-llms-ollama

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
import pandas as pd
from llama_index.core.prompts import PromptTemplate
from llama_index.core import download_loader, VectorStoreIndex, ServiceContext
from langchain.embeddings import HuggingFaceInstructEmbeddings

from IPython.display import display, HTML
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.schema import QueryBundle
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.llms.ollama import Ollama

# Setup

1. In this section we will work with the LLM overview paper paper and create an initial set of nodes (chunk size 500).
2. We will use Open Source LLM from ollama

# Load Data

In [3]:
PDFReader = download_loader("PDFReader")
loader = PDFReader()
docs = loader.load_data(file=Path("data/LLM_Overview_Paper.pdf"))

/var/folders/q3/ktj7fz113851fp741w1x3rf00000gn/T/ipykernel_47599/2854157064.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PDFReader = download_loader("PDFReader")


In [4]:
from llama_index.core.node_parser import SimpleNodeParser
node_parser = SimpleNodeParser.from_defaults(chunk_size=1000)
nodes = node_parser.get_nodes_from_documents(docs)

In [24]:
len(nodes)

119

# Models

## LLM - Ollama

In [21]:
cohere_api_key = "REPLACE WITH YOUR COHERE_API_KEY"

llm = Ollama(model="mistral")

## Embedding Model

In [19]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5')


/Users/aboniasojasingarayar/miniconda3/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


## Configure Index and Retriever

In [20]:
# ServiceContext
service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model=embed_model
                                               )

# index
vector_index = VectorStoreIndex(
    nodes, service_context=service_context
)

# configure retriever
retriever = VectorIndexRetriever(
    index=vector_index,
    similarity_top_k=10,
    service_context=service_context)

/var/folders/q3/ktj7fz113851fp741w1x3rf00000gn/T/ipykernel_47599/4172195817.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm,


## Initialize Re-rankers

In [22]:
# Define all embeddings and rerankers
RERANKERS = {
    "WithoutReranker": "None",
    "CohereRerank": CohereRerank(api_key=cohere_api_key, top_n=5),
    "bge-reranker-base": SentenceTransformerRerank(model="BAAI/bge-reranker-base", top_n=5),
    "bge-reranker-large": SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=5)
}

## Retrieval Comparisons

In [36]:
import pandas as pd
from IPython.display import display, HTML
from copy import deepcopy

# Assuming QueryBundle and retriever are defined elsewhere
# Assuming RERANKERS is a dictionary of rerankers

def retrieve_and_process_nodes(query_str, reranker):
    query_bundle = QueryBundle(query_str)
    retrieved_nodes = retriever.retrieve(query_bundle)
    if reranker != "None":
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    return retrieved_nodes

def get_retrieved_nodes(query_str, reranker):
    return retrieve_and_process_nodes(query_str, reranker)

def pretty_print(df):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    display(HTML(df.to_html().replace("\\n", "<br>")))

def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = [{"Score": node.score, "Text": node.node.get_text().replace("\n", " ")} for node in nodes]
    pretty_print(pd.DataFrame(result_dicts))

In [37]:
query_str = "What are theLLM finetuning Methods mentionned in the paper?"

for rerank_name, reranker in RERANKERS.items():
    print(f"Running Evaluation for Reranker: {rerank_name}")
    retrieved_nodes = retrieve_and_process_nodes(query_str, reranker)
    print(f"Visualize Retrieved Nodes for Reranker: {rerank_name}")
    visualize_retrieved_nodes(retrieved_nodes)

Running Evaluation for Reranker: WithoutReranker
Visualize Retrieved Nodes for Reranker: WithoutReranker


Running Evaluation for Reranker: CohereRerank
Visualize Retrieved Nodes for Reranker: CohereRerank


Running Evaluation for Reranker: bge-reranker-base


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Visualize Retrieved Nodes for Reranker: bge-reranker-base


Running Evaluation for Reranker: bge-reranker-large


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Visualize Retrieved Nodes for Reranker: bge-reranker-large


# Evaluation

Now, we will use RetrieverEvaluator to evaluate the quality of any Retriever module.

# Evaluation metrics:

1. hit-rate 
2. MRR. 


## Build an Evaluation dataset of (query, context) pairs
Here we build a simple evaluation dataset over the existing text corpus.

We use our generate_question_context_pairs to generate a set of (question, context) pairs over a given unstructured text corpus. This uses the LLM to auto-generate questions from each context chunk.We will use `Mistral` LLM to generate Question-Context Pairs.

We get back a EmbeddingQAFinetuneDataset object. At a high-level this contains a set of ids mapping to queries and relevant doc chunks, as well as the corpus itself.

In [7]:
qa_generate_prompt_tmpl = """\
Context:

---------------------
{context_str}
---------------------

Task: As a Professor, your objective is to create {num_questions_per_chunk} questions for an upcoming quiz or examination. The questions should be varied and cover the provided context thoroughly. They should not include options or start with Q1/Q2. Ensure the questions are restricted to the context information provided.
"""

In [7]:
import random
nodes_filter = random.sample(nodes, 5)
# Evaluator
from llama_index.core.evaluation import (
    generate_question_context_pairs,
)
from llama_index.core.evaluation import RetrieverEvaluator
llm = Ollama(model="mistral", request_timeout=300.0)

qa_dataset = generate_question_context_pairs(
    nodes_filter, llm=llm, num_questions_per_chunk=2, qa_generate_prompt_tmpl=qa_generate_prompt_tmpl
)

100%|██████████| 5/5 [02:36<00:00, 31.30s/it]


In [8]:
len(qa_dataset.corpus.keys())

5

In [10]:
qa_dataset

EmbeddingQAFinetuneDataset(queries={'543f75c8-5dce-49df-9d65-c6928363da64': 'Question 1: Which techniques were used in the training of UL2 model to improve its performance for downstream tasks? please specify the denoisers and their respective percentages.', '93567bf7-86e6-43e0-b0c9-db3d83d93c01': 'Question 2: Compare and contrast the training methods of PaLM-2, 1.20 U-PaLM, and GLM-130B in terms of their pre-training objectives, architecture, and improvements over baseline models on various NLP tasks.', '71b9a7a7-573b-49d1-886a-e0d7fe2736d2': 'Which pre-trained language models (LLMs) have been employed for question-answering (QA), classification (Clf), natural language inference (NLI), machine translation (MT), reading comprehension (RC), commonsense reasoning (CR), mathematical reasoning (MR), and memorization (Mem.) tasks, as illustrated in Table X?', '19bcc273-e9ec-4ee4-a4fb-f85365637705': 'In which benchmark is the BIG-benchmark model with SuperGLUE used for evaluating QA, Clf, NL

In [14]:
# Generated 2 questions for this chunk
qa_dataset.queries['9596503c-800b-44e9-bc52-530501823fe6']

'How do the approaches presented in "Bloom," "OPT," "Palm," "Scaling instruction-finetuned language models," and "Multitask prompted training" differ from each other in terms of scaling language modeling?'

In [15]:
# Extract relevant doc for this chunk
qa_dataset.relevant_docs['9596503c-800b-44e9-bc52-530501823fe6']

['25372f52-a0a3-4409-b8f5-0c429e4e073d']

In [16]:
# Extract corpus for this relevant doc
qa_dataset.corpus['25372f52-a0a3-4409-b8f5-0c429e4e073d']

', “Bloom: A 176b-\nparameter open-access multilingual language model,” arXiv preprint\narXiv:2211.05100 , 2022. 2, 4, 9, 10, 20, 21, 23, 27\n[14] S. Zhang, S. Roller, N. Goyal, M. Artetxe, M. Chen, S. Chen,\nC. Dewan, M. Diab, X. Li, X. V . Lin et al. , “Opt: Open pre-trained\ntransformer language models,” arXiv preprint arXiv:2205.01068 , 2022.\n2, 8, 10, 21, 23\n[15] A. Chowdhery, S. Narang, J. Devlin, M. Bosma, G. Mishra, A. Roberts,\nP. Barham, H. W. Chung, C. Sutton, S. Gehrmann et al. , “Palm: Scaling\nlanguage modeling with pathways,” arXiv preprint arXiv:2204.02311 ,\n2022. 2, 6, 9, 10, 20, 21, 22, 23\n[16] H. W. Chung, L. Hou, S. Longpre, B. Zoph, Y . Tay, W. Fedus, E. Li,\nX. Wang, M. Dehghani, S. Brahma et al. , “Scaling instruction-finetuned\nlanguage models,” arXiv preprint arXiv:2210.11416 , 2022. 2, 6, 7, 12,\n13, 15, 20, 21, 23, 25, 28\n[17] V . Sanh, A. Webson, C. Raffel, S. H. Bach, L. Sutawika, Z. Alyafeai,\nA. Chaffin, A. Stiegler, T. L. Scao, A. Raja et al. , “Mul

In [ ]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[1]
sample_expected = qa_dataset.relevant_docs[sample_id]
#sample_expected = qa_dataset.corpus['4d7593c4-85b5-420d-944d-1ffe07bacf0f']

retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

In [38]:
sample_query

'Question 2: Compare and contrast the training methods of PaLM-2, 1.20 U-PaLM, and GLM-130B in terms of their pre-training objectives, architecture, and improvements over baseline models on various NLP tasks.'

In [25]:
def get_sample_data(qa_dataset):
    """
    Extracts a sample query and its expected results from the QA dataset.
    
    Args:
        qa_dataset (object): The question-answering dataset object.
        
    Returns:
        tuple: A tuple containing the sample ID, sample query, and expected results.
    """
    # Extracting the first item from the dataset's queries
    sample_id, sample_query = list(qa_dataset.queries.items())[0]
    sample_expected = qa_dataset.relevant_docs[sample_id]
    return sample_id, sample_query, sample_expected

def evaluate_retriever(retriever, sample_query, sample_expected):
    """
    Evaluates a retriever model against a sample query using specified metrics.
    
    Args:
        retriever (object): The retriever model to evaluate.
        sample_query (str): The sample query to evaluate.
        sample_expected (list): The expected results for the sample query.
        
    Returns:
        dict: The evaluation results, including MRR and Hit Rate.
    """
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    try:
        eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
        return eval_result
    except Exception as e:
        print(f"Evaluation failed: {e}")
        return None

def main():
    # Assuming qa_dataset and retriever are already defined
    sample_id, sample_query, sample_expected = get_sample_data(qa_dataset)
    eval_result = evaluate_retriever(retriever, sample_query, sample_expected)
    if eval_result:
        print(eval_result)

if __name__ == "__main__":
    main()


Evaluation failed: asyncio.run() cannot be called from a running event loop


/var/folders/q3/ktj7fz113851fp741w1x3rf00000gn/T/ipykernel_47599/582604229.py:36: RuntimeWarning: coroutine 'BaseRetrievalEvaluator.aevaluate' was never awaited
  return None
